In [1]:
import torch
import torch.nn as nn
import torch.optim as optim


# 定义一个LSTM模型
class SoilMoistureRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(SoilMoistureRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # LSTM层
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

        # 全连接层，将LSTM的输出映射到土壤含水量预测
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # 初始化LSTM的隐藏状态和细胞状态
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # 前向传播LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: (batch_size, seq_length, hidden_size)

        # 解码LSTM的最后一个时间步的输出
        out = self.fc(out[:, -1, :])  # 取最后一个时间步的输出
        return out


# 设置输入、隐藏和输出的维度
input_size = 4  # 输入特征数量（如降水量、土壤含水量、长波时延、地表温度）
hidden_size = 64  # LSTM的隐藏层大小
num_layers = 2  # 堆叠的LSTM层数
output_size = 3  # 输出特征数量（预测多少天的土壤含水量）

# 创建模型实例
model = SoilMoistureRNN(input_size, hidden_size, output_size, num_layers)

# 定义损失函数和优化器
criterion = nn.MSELoss()  # 均方误差
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 打印模型架构
print(model)


SoilMoistureRNN(
  (lstm): LSTM(4, 64, num_layers=2, batch_first=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
)
